In [1]:
from bs4 import BeautifulSoup
import requests
import time
import os
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
path="C:\Program Files (x86)\msedgedriver.exe"

<>:10: SyntaxWarning: invalid escape sequence '\P'
<>:10: SyntaxWarning: invalid escape sequence '\P'
C:\Users\9616k\AppData\Local\Temp\ipykernel_17116\1646112386.py:10: SyntaxWarning: invalid escape sequence '\P'
  path="C:\Program Files (x86)\msedgedriver.exe"


In [2]:
BASE_URL="https://xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h"
EXTENSION="/energy"

In [3]:
import re

def clean_name(folder_name):
    # Replace special characters in folder name with '-'
    # For special cases like while scraping data for category: Guidelines for Release/Notification, Provisional Notification and De-notification of Cultivars
    cleaned_name = re.sub(r'[\/:*?"<>| ]', '-', folder_name)
    cleaned_name = cleaned_name.replace("\"", "")
    cleaned_name = cleaned_name.replace("'", "")
    return cleaned_name


In [4]:
def fetchData(soup):
    x=0
    MiddleColumn = soup.find('div', id='MiddleColumn')
    MiddleColumn = soup.find('div', id='texttospeak')
    if MiddleColumn:
        # Remove all tags and add only text
        text_content = ''
        for element in MiddleColumn.contents:
            x+=1
            if element.name == 'h3' or element.name == 'h4' or element.name == 'p':
                text_content += str(element.text) + '\n'
            else:
                text_content += str(element)
    if x<3:
        MiddleColumn = soup.find('div', id='MiddleColumn_internal')
        # Remove all <a> tags from MiddleColumn
        for a_tag in MiddleColumn.find_all('a'):
            a_tag.decompose()
        # Remove all tags and add only text
        text_content = ''
        for element in MiddleColumn.contents:
            if element.name == 'h3' or element.name == 'h4' or element.name == 'p':
                text_content += str(element.text) + '\n'
            else:
                text_content += str(element)
    # Remove HTML tags using regular expression
    return re.sub(r'<.*?>', '', text_content)

In [5]:
def webScrapeSubCategory(url, folder_path='.'):
    #For Delay
    try:
        data = requests.get(url, verify=False) #Ignoring SSL certificate verification
        soup = BeautifulSoup(data.text, "html.parser") # Generating Soup
#         print(soup)
        a_tags = soup.find_all('a', class_='folderfile_name') #Fetching a tags
        result = []
        if a_tags:
            for tag in a_tags:
                category_name = tag.text.strip() #removing extra spaces or lines
                category_folder_name = clean_name(category_name) #clean name
                category_folder_path = os.path.join(folder_path, category_folder_name) # calulate path
                os.makedirs(category_folder_path, exist_ok=True) # If directory does not exitst create one
                print(f"Scraping data for category: {category_name} {url}") #Show message --> For Debugging
                category_url = BASE_URL + tag.get('href') #Next website
                subcategories = webScrapeSubCategory(category_url, category_folder_path) # recursively call
                result.append({category_name: subcategories})
        else:
            print(f"Fetching content for {url}")
            driver = webdriver.Edge() 
            driver.get(url)
            time.sleep(5)
            html_content = driver.page_source
            driver.quit() 
            soup = BeautifulSoup(html_content, "html.parser") # Generating Soup
            title = soup.find('h3', class_='title')
            if title is None:
                title = soup.find('h3')
            scrapped_data = title.text + "\n\n" + fetchData(soup)     
            with open(os.path.join(folder_path, clean_name(title.text)+'.txt'), 'w', encoding='utf-8') as file:
                file.write(scrapped_data)
            return scrapped_data
        return result
    except Exception as e:
        print("EXCEPTION OCCURRED",e)
        return ""


In [10]:
def webScrape(url, start,end,folder_path='.'):
    #For Delay
    try:
        data = requests.get(url, verify=False) #Ignoring SSL certificate verification
        soup = BeautifulSoup(data.text, "html.parser") # Generating Soup
        a_tags = soup.find_all('a', class_='folderfile_name') #Fetching a tags
        result = []
        if a_tags:
            for tag in a_tags:
                category_name = tag.text.strip() #removing extra spaces or lines
                category_folder_name = clean_name(category_name) #clean name
                category_folder_path = os.path.join(folder_path, category_folder_name) # calulate path
                os.makedirs(category_folder_path, exist_ok=True) # If directory does not exitst create one
                print(f"Scraping data for category: {category_name} {url}") #Show message --> For Debugging
                category_url = BASE_URL + tag.get('href') #Next website
                subcategories = webScrapeSubCategory(category_url, category_folder_path) # recursively call
                result.append({category_name: subcategories})
        else:
            title = soup.find('h3', class_='card-title title')
            scrapped_data = title.text + "\n\n" + fetchData(url)
            with open(os.path.join(folder_path, clean_name(title.text)+'.txt'), 'w', encoding='utf-8') as file:
                file.write(scrapped_data)
            return scrapped_data
    
        return result
    except Exception as e:
        print("EXCEPTION OCCURRED",e)

In [11]:
#For generating list with indices
data = requests.get(BASE_URL+EXTENSION, verify=False) #Ignoring SSL certificate verification
soup = BeautifulSoup(data.text, "html.parser") # Generating Soup
a_tags = soup.find_all('a', class_='folderfile_name') #Fetching a tags
# print(soup)
result = []
if a_tags:
    for i,tag in enumerate(a_tags):
        result.append([tag.text,i])
    print(result)
else:
    print("No a tags found")

[['அரசு திட்டங்களும் கொள்கைகளும்', 0], ['எரிசக்தி அடிப்படை', 1], ['எரிசக்தி உற்பத்தி', 2], ['எரிசக்தி சேமிப்பு', 3], ['எரிசக்தி திறன்', 4], ['சிறந்த செயல்முறைகள்', 5], ['சுற்றுச்சூழல்', 6], ['தொிந்து கொள்ள வேண்டிய எரிசக்தி சார்ந்த தகவல்கள்', 7], ['பெண்களும் எரிசக்தியும்', 8]]


C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
allSectors=webScrape(BASE_URL+EXTENSION,0,0,"Tamil/energy") #START AND END ARE BOTH INCLUDING INDICES

C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Scraping data for category: அரசு திட்டங்களும் கொள்கைகளும் https://xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h/energy


C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetching content for https://xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h/energy/அரசு-திட்டங்களும்-கொள்கைகளும்
Scraping data for category: எரிசக்தி அடிப்படை https://xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h/energy


C:\Users\9616k\Documents\CODES\WebScrapLearning\venv\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetching content for https://xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h/energy/எரிசக்தி-அடிப்படை
Scraping data for category: எரிசக்தி உற்பத்தி https://xn--clcu6av0at5becfj8m.xn--xkc2dl3a5ee0h/energy


# FLATTEN

In [45]:
import os
import shutil
from pathlib import Path

In [46]:
def move(source,dest):
    isExist = os.path.exists(source)
    os.makedirs(dest, exist_ok=True)
    if isExist is False:
        return "PATH DOES NOT  EXIST"
    try:
        for item in os.listdir(source):
            item_path = os.path.join(source, item)
            if os.path.isdir(item_path):
                move(item_path,dest)  # Recursively call move function for directories
            else:
                # print(item)
                with open(os.path.join(os.getcwd(),item_path),"r",encoding="utf-8") as c:
                    with open(os.path.join(os.getcwd(),dest,clean_name(item)),"w",encoding="utf-8") as nf:
                        nf.write(c.read())
                # print(f"SUCCESS FOR {item}")
    except Exception as e:
        print(e)

In [49]:
source="SINDHI"
for category in os.listdir(source):
    print(category)

AGRICULTURE
e-governance
health


In [52]:
source="SINDHI/health"
dest=source+"_FLATTEND"
# dest="VIKASPEDIA_DATA_SOCIAL_WELFARE_FLATTENED"
for item in os.listdir(source):
    item_path = os.path.join(source, item)
    if os.path.isdir(item_path):
            move(item_path, os.path.join(dest, item))  # Recursively call move function for directories

# INSIGHTS

In [79]:
import os

source = "DOHNGRI"
tot_folders = 0
tot_files = 0
tot_empty = 0

for item in os.listdir(source):
    folder_path = os.path.join(source, item)
    if item.endswith("_FLATTENED") and os.path.isdir(folder_path):
        sub_folders = os.listdir(folder_path)
        tot_folders += len(sub_folders)  # Accumulate total sub-folders
        
        # Count files in each folder
        for sub_folder in sub_folders:
            sub_folder_path = os.path.join(folder_path, sub_folder)
            if os.path.isdir(sub_folder_path):
                files_in_sub_folder = os.listdir(sub_folder_path)
                if not files_in_sub_folder:
                    tot_empty += 1  # Increment empty folder count
                tot_files += len(files_in_sub_folder)  # Accumulate total files
                print(sub_folder, len(files_in_sub_folder))

print("----------------------------------")
print("Total Folders:", tot_folders)
print("Total Empty Folders:", tot_empty)
print("Total Files:", tot_files)


----------------------------------
Total Folders: 0
Total Empty Folders: 0
Total Files: 0


In [78]:
import os

source = "MANIPURI"
tot_folders = 0
tot_files = 0
tot_empy = 0

for item in os.listdir(source):
    folder_path = os.path.join(source, item)
    if item.endswith("_FLATTEND") and os.path.isdir(folder_path):
        sub_folders = os.listdir(folder_path)
        tot_folders=len(sub_folders)
        # Count files in each folder
        for i in sub_folders:
            x=len(os.listdir(os.path.join(source,item,i)))
            if x==0:
                tot_empy+=1
            else:
                tot_files+=x
            print(i,x)
print("----------------------------------")
print("Total Folders:", tot_folders)
print("Total Empty Folders:", tot_empy)
print("Total Files:", tot_files)

এগ্রি-ইন্সুরেন্স 1
এগ্রি-ক্রেদিট 0
এগ্রী-ইনপুটশীং 3
পোলিসিশীং-অমসুং-স্কিমশীং 6
মার্কেটকী-ইনফোর্মেসন 2
লাইভষ্টোক 0
VLEশীংগী-রিসোর্সশীং 2
ই-গভর্নান্স-সর্ভিসেস-ওনলাইন 1
দিজিটেল-ইন্দিয়া 0
দিজিটেল-পেমেন্ট 5
নাগরিক্কী-ফিদবেক্কীদমক্তা-মোবাইল-এপশীং 0
ফার্মা-জন-সমাধান-(ই-গবর্নান্স) 1
ভারতকী-জুদিসিএল-সর্ভিসশীং 1
ভারত্তা-ই-গভর্নান্স 2
মোবাইল-গবর্নান্স 0
সিটিজেন-এক্সেস-তু-ইনফোর্মেসন-(RTI) 2
সিটিজেনগী-সর্ভিসশীং 2
অঙাংগী-হকশীং 3
অঙাংশীংগী-কোর্নর 2
কেরিয়র-গাইদেন্স 1
দিজিটেল-লিটরেসি 10
পেরেন্টশীংগী-কোর্ণর 1
পোলিসিশীং-অমসুং-স্কিমশীং 4
এনভাইরনমেন্ট 1
এনর্জি-এক্সেস 2
এনর্জি-য়োকপা 0
এনর্জিগী-বেসিকশীং 1
পোলিসি-সপোর্ট 3
আয়ুশ 1
এজিং(অহল-ওইরকপা) 0
দিজিজ-অমসুং-দিসওর্দরশীং 0
নুপীগী-হকশেল 5
ন্যুত্রিসন 1
পুন্সিগী-ওইবা-অচুম্বশীং 14
পোলিসিশীং-অমসুং-স্কিমশীং 5
ফার্ষ্ট-এইদ 6
সেনিটেসন-অমসুং-হায়জিন 4
হকশেলগী-কেম্পেইনশীং 1
হেলথ-কেয়র-ইনোভেসনশীং 1
অর্বান-পোবর্টি-এলেভিএসন 0
এন্ত্রিপ্রেন্যুওরশিপ 3
ওর্গানাইজ-তৌদবা-সেক্টর-ৱেলফেয়র 6
খুঙ্গংগী-লায়রবা-কোকহনবা 8
দিজাষ্টর-মেনেজমেন্ট 15
নুপী-অমদি-অঙাংশীংগী-চাউখৎ-থৌরাং-পায়খৎপা 13
ফাই

In [72]:
import os

source = "SINDHI"
tot_folders = 0
tot_empy = 0

for item in os.listdir(source):
    folder_path = os.path.join(source, item)
    if item.endswith("_FLATTEND") and os.path.isdir(folder_path):
        sub_folders = os.listdir(folder_path)
        tot_folders=len(sub_folders)
        # Count files in each folder
        for i in sub_folders:
            x=len(os.listdir(os.path.join(source,item,i)))
            if x==0:
                tot_empy+=1
            print(i,x)
print("Total Folders:", tot_folders)
print("Total Empty Folders:", tot_empy)

ऐगरी-बिमो 1
कोकिड 0
चौपायो-माल 0
निती-ऐं-रिथाउं 1
पोख-इंपुट 0
मच्छी-पालन 0
डि॒जिटल-माली-सेवायूं 1
आयूश 0
खोराक 0
जाईफाणी-सिहत 0
बा॒राणा-हक 0
बिमारी 18
Total Folders: 5
Total Empty Folders: 8
